# Calculating differences between World Ocean Atlas 2023 and GFDL-MOM6-COBALT2 model outputs
**Author:** Denisse Fierro Arcos  
**Date:** 2024-10-03  
  
We use the original GFDL-MOM6-COBALT2 model outputs for ocean temperature (`thetao`) and salinity (`so`) to calculate a climatology mean matching the period covered by World Ocean Atlas 2023 (WOA23): 1981-2010. These climatologies are saved as `zarr` files for use in the future.

We calculate the following:
- GFDL outputs are substracted from WOA23 data
- Percentage difference betwen GFDL and WOA23 data

Both products are saved as `parquet` files at a global scale, and data is also extracted and saved for each FishMIP regional model.

In [1]:
import xarray as xr
import os
from glob import glob
import matplotlib.pyplot as plt
from dask.distributed import Client
import geopandas as gpd
import numpy as np
import pandas as pd

## Starting a cluster
This will allow us to automatically parallelising tasks on large datasets.

In [2]:
client = Client(threads_per_worker = 1)

In [3]:
base_folder = '/g/data/vf71/fishmip_inputs/ISIMIP3a/global_inputs/obsclim/025deg'
out_folder = os.path.join(base_folder, 'comp_clim_woa')
os.makedirs(out_folder, exist_ok = True)

## Defining function calculating climatology from WOA data

In [4]:
def gfdl_match_woa(file_path, file_out, monthly = False):
    '''
    Inputs:
    file_path (character): Full file path where original GFDL netCDF file is stored.
    file_out (character): Full file path where climatology will be stored.
    monthly (boolean). Default is False. If set to True, then monthly climatologies 
    are calculated.

    Outputs:
    da (data array): Climatological mean matching WOA period: 1981-2010
    '''
    da = xr.open_dataarray(file_path)
    
    #Select same period as WOA data
    da = da.sel(time = slice('1981', '2010'))
    
    #Rechunk data and rename depth variable to match WOA data
    da = da.chunk({'lat': 144, 'lon': 288}).rename({'lev': 'depth'})
    
    #Calculate climatology mean
    if monthly:
        da = da.groupby('time.month').mean('time')
    else:
        da = da.mean('time')
    
    #Save results as zarr file
    da.to_zarr(file_out, consolidated = True, mode = 'w')

    return da

## Defining location of original GFDL files and output paths

In [5]:
#Temperature files
gfdl_temp_file = os.path.join(
    base_folder, 'gfdl-mom6-cobalt2_obsclim_thetao_15arcmin_global_monthly_1961_2010.nc')
gfdl_out_temp = os.path.join(
    out_folder,  'gfdl-mom6-cobalt2_obsclim_global_clim_mean_temp_1981_2010.zarr')

#Salinity files
gfdl_sal_file = os.path.join(
    base_folder, 'gfdl-mom6-cobalt2_obsclim_so_15arcmin_global_monthly_1961_2010.nc')
gfdl_out_sal = os.path.join(
    out_folder, 'gfdl-mom6-cobalt2_obsclim_global_clim_mean_sal_1981_2010.zarr')

## Applying function

In [6]:
gfdl_temp = gfdl_match_woa(gfdl_temp_file, gfdl_out_temp)
gfdl_sal = gfdl_match_woa(gfdl_sal_file, gfdl_out_sal)

## *Optional: Loading GFDL climatologies matching WOA period*
If climatologies were already calculated, you can upload them using the chunk below instead of recalculating them.

In [6]:
gfdl_temp = xr.open_zarr(gfdl_out_temp).thetao
gfdl_sal = xr.open_zarr(gfdl_out_sal).so

## Loading regridded World Ocean Atlas (WOA)
This step was performed in the `regridding_woa_data.ipynb` script.

In [7]:
base_woa = '/g/data/vf71/WOA_data/global'
temp_woa = xr.open_zarr(
    os.path.join(base_woa, 
                 'regridded_woa_clim_mean_temp_1981-2010.zarr')).temperature
salt_woa = xr.open_zarr(
    os.path.join(base_woa, 
                 'regridded_woa_clim_mean_sal_1981-2010.zarr')).salinity

## Calculating difference between GFDL and WOA

In [8]:
diff_temp = gfdl_temp-temp_woa
per_diff_temp = (diff_temp/temp_woa)*100

diff_sal = gfdl_sal-salt_woa
per_diff_sal = (diff_sal/salt_woa)*100

## Loading FishMIP regions mask and shapefile

In [9]:
# Define folder with shared resources
shared_res = '/g/data/vf71/shared_resources'
# Loading shapefile
rmes = gpd.read_file(
    os.path.join(shared_res, 'FishMIP_regional_models/FishMIP_regional_models.shp'))

# Loading mask
mask_ras = xr.open_dataset(
    os.path.join(shared_res, 
                 'FishMIPMasks/merged_regional_fishmip',
                 'gfdl-mom6-cobalt2_areacello_15arcmin_fishMIP_regional_merged.nc')).region
#Renaming coordinate dimensions
mask_ras = mask_ras.rename({'latitude': 'lat', 'longitude': 'lon'})
#Rechunking data to make it more manageable
mask_ras = mask_ras.chunk({'lat': 144, 'lon': 288})

## Defining functions to mask and extract data

In [10]:
def mask_data(da, mask):
    '''
    Open netCDF files in analysis ready data (ARD) format. That is apply chunks
    that make data analysis easier.
    
    Inputs:
    da (data array): Data array containing data to be extracted
    mask_ras (boolean data array): Data array to be used as initial mask
    to decrease the size of the original dataset. This mask makes no distinction
    between regional models, it simply identifies grid cells within regional 
    model boundaries with the value of 1.
    
    Outputs:
    da (data array): ARD data array containing data only for grid cells within
    regional model boundaries.
    '''
    
    da_mask = da.where(mask == 1)
    da_mask.rio.set_spatial_dims(x_dim = 'lon', y_dim = 'lat', inplace = True)
    da_mask.rio.write_crs('epsg:4326', inplace = True)
    da_mask = da_mask.chunk({'lat': 144, 'lon': 288})
    return da_mask

In [64]:
def clip_data(da, region, file_out):
    '''
    Open netCDF files in analysis ready data (ARD) format. That is apply chunks
    that make data analysis easier.
    
    Inputs:
    da (data array): Data array containing data to be extracted
    region (shapefile): Shapefile containing the boundaries of regional models
    file_out (character): Full file path where masked data should be stored.
    
    Outputs:
    No data is returned, but masked file will be stored in specified file path.
    '''
    #Clip data using regional shapefile
    da_mask = da.rio.clip(region.geometry, region.crs, drop = True, 
                          all_touched = True)
    #Remove spatial information
    da_mask = da_mask.drop_vars('spatial_ref')
    da_mask.encoding = {}

    #Keep data array attributes to be recorded in final data frame
    da_attrs = da.attrs
    da_attrs = pd.DataFrame([da_attrs])
    #Set column order
    if 'month' in list(da_mask.coords):
        ind_wider = ['lat', 'lon', 'month', 'depth', 'vals']
    else:
        ind_wider = ['lat', 'lon', 'depth', 'vals']
    #Turn extracted data into data frame and remove rows with NA values
    df = da_mask.to_series().to_frame().reset_index().dropna()
    #Changing column name to standardise across variables
    df = df.rename(columns = {da.name: 'vals'}).reset_index(drop = True)
    #Reorganise data
    df = df[ind_wider]
    #Include original dataset attributes
    df = pd.concat([df, da_attrs], axis = 1)
    #Saving data frame
    df.to_parquet(file_out)

## Extracting regridded WOA data for each region

In [12]:
base_out = '/g/data/vf71/fishmip_inputs/ISIMIP3a/regional_inputs/obsclim/025deg/comp_maps'
os.makedirs(base_out, exist_ok = True)

In [14]:
diff_temp_masked = mask_data(diff_temp, mask_ras)
diff_temp_masked.name = 'temperature'

per_diff_temp_masked = mask_data(per_diff_temp, mask_ras)
per_diff_temp_masked.name = 'temperature'

for i in rmes.region:
    #Get polygon for each region
    mask = rmes[rmes.region == i]
    #Get name of region and clean it for use in output file
    reg_name = mask['region'].values[0].lower().replace(" ", "-").replace("'", "")
    #File name out - Replacing "global" for region name
    file_out_diff = f'diff_gfdl-woa_{reg_name}_temp_1981-2010.parquet'
    file_out_per_diff = f'per_diff_gfdl-woa_{reg_name}_temp_1981-2010.parquet'
    clip_data(diff_temp_masked, mask, os.path.join(base_out, file_out_diff))
    clip_data(per_diff_temp_masked, mask, os.path.join(base_out, file_out_per_diff))

In [15]:
diff_sal_masked = mask_data(diff_sal, mask_ras)
diff_sal_masked.name = 'salinity'

per_diff_sal_masked = mask_data(per_diff_sal, mask_ras)
per_diff_sal_masked.name = 'salinity'

for i in rmes.region:
    #Get polygon for each region
    mask = rmes[rmes.region == i]
    #Get name of region and clean it for use in output file
    reg_name = mask['region'].values[0].lower().replace(" ", "-").replace("'", "")
    #File name out - Replacing "global" for region name
    file_out_diff = f'diff_gfdl-woa_{reg_name}_sal_1981-2010.parquet'
    file_out_per_diff = f'per_diff_gfdl-woa_{reg_name}_sal_1981-2010.parquet'
    clip_data(diff_sal_masked, mask, os.path.join(base_out, file_out_diff))
    clip_data(per_diff_sal_masked, mask, os.path.join(base_out, file_out_per_diff))

## Defining location of monthly output paths

In [13]:
#Temperature files
gfdl_out_temp_month = os.path.join(
    out_folder,  'gfdl-mom6-cobalt2_obsclim_global_monthly_clim_mean_temp_1981_2010.zarr')

#Salinity files
gfdl_out_sal_month = os.path.join(
    out_folder, 'gfdl-mom6-cobalt2_obsclim_global_monthly_clim_mean_sal_1981_2010.zarr')

## Applying function
This only needs to be done once for the entire globe. If new FishMIP models are submitted, data can be loaded in the next chunk.

In [6]:
gfdl_temp_month = gfdl_match_woa(gfdl_temp_file, gfdl_out_temp_month, 
                                 monthly = True)
gfdl_sal_month = gfdl_match_woa(gfdl_sal_file, gfdl_out_sal_month,
                                monthly = True)

INFO:flox:Entering _validate_reindex: reindex is None
INFO:flox:Leaving _validate_reindex: method = None, returning None
INFO:flox:_choose_engine: Choosing 'numpy'
INFO:flox:_choose_method: method is None
INFO:flox:_choose_method: choosing preferred_method=blockwise
INFO:flox:Entering _validate_reindex: reindex is None
INFO:flox:Leaving _validate_reindex: reindex is False


## *If monthly climatologies were calculated*: Load monthly climatologies

In [34]:
#Masking temperature and salinity monthly climatologies
gfdl_temp_month = xr.open_zarr(gfdl_out_temp_month).thetao
gfdl_temp_month_masked = mask_data(gfdl_temp_month, mask_ras)

gfdl_sal_month = xr.open_zarr(gfdl_out_sal_month).so
gfdl_sal_month_masked = mask_data(gfdl_sal_month, mask_ras)

In [66]:
#Location of folder where monthly climatology per region will be stored
folder_out = os.path.join('/g/data/vf71/fishmip_inputs/ISIMIP3a/regional_inputs',
                          'obsclim/025deg/monthly_clim_corr')

for i in rmes.region:
    #Get polygon for each region
    mask = rmes[rmes.region == i]
    #Get name of region and clean it for use in output file
    reg_name = mask['region'].values[0].lower().replace(" ", "-").replace("'", "")
    #File name out - Replacing "global" for region name
    file_out_temp = f'gfdl-mom6-cobalt2_obsclim_thetao_15arcmin_{reg_name}_mthly_clim_mean_1981_2010.parquet'
    file_out_sal = f'gfdl-mom6-cobalt2_obsclim_so_15arcmin_{reg_name}_mthly_clim_mean_1981_2010.parquet'
    clip_data(gfdl_temp_month_masked, mask, 
              os.path.join(folder_out, file_out_temp))
    clip_data(gfdl_sal_month_masked, mask, 
              os.path.join(folder_out, file_out_sal))